In [4]:
import cv2
import numpy as np

In [5]:
#Laoding a YOLO v3 - Pretrained Model

def reject_majority(image_path):
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    
    #loading names of the classes from Microsoft COCO Dataset
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f]
    
    #Take the input image
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    total_area = height * width
    
    #Detectht the objects
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    flag = False
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
    
            #Compute Area taken
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                area = w * h
                
                if area / total_area > 0.3:
                    flag = True
                    # cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
                    # cv2.putText(image, f"{classes[class_id]} - {confidence:.2f}", (x, y - 5),
                    #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    # # Display image with segmentation
    # cv2.imshow("Image", image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    return not flag

In [7]:
reject_majority("cat.jpg")

False